In [1]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from datasets import load_dataset 

dataset = load_dataset("glue", "sst2")

In [20]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [21]:
def func(x):
    return tokenizer(x['sentence'], padding="max_length", truncation=True, max_length=128)
tokenized_dataset = dataset.map(func, batched=True)

In [22]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [23]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

trainer.train()

  0%|          | 0/12630 [00:00<?, ?it/s]

{'loss': 0.4814, 'grad_norm': 9.369991302490234, 'learning_rate': 1.9208234362628665e-05, 'epoch': 0.12}
{'loss': 0.3338, 'grad_norm': 9.176271438598633, 'learning_rate': 1.8416468725257326e-05, 'epoch': 0.24}
{'loss': 0.2927, 'grad_norm': 6.52882194519043, 'learning_rate': 1.762470308788599e-05, 'epoch': 0.36}


KeyboardInterrupt: 